In [ ]:
import numpy as np
import pandas as pd
#import fiona 
#import math
#import xarray as xa
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
#import datetime
#import random
#from scipy.interpolate import interp1d
import geopandas as gpd

In [ ]:
gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2e.gdb'
hrus_v10_path = 'c:/Users/markstro/work/GIS/hrusAllConus/hrusAllConus.shp'
farns_v10_path = 'c:/Users/markstro/work/farnsworth/'

In [ ]:
# Read the version 1.0 shapefile
hrus_v10 = gpd.read_file(hrus_v10_path)

In [ ]:
coolwarm = cm.get_cmap('coolwarm', 12)
print(coolwarm)

In [ ]:
hrus_v10.plot()

In [ ]:
print(hrus_v10.shape)

In [ ]:
hrus_v10.head()

In [ ]:
shape_nhm_id = hrus_v10["nhm_id"].values
print(shape_nhm_id)
print(len(shape_nhm_id))
print(min(shape_nhm_id), max(shape_nhm_id))

In [ ]:
# Read the Farnsworth PET data for version 1.0
# Units of values in these files is mm/month

fns = ["farnsPET_r01.csv", "farnsPET_r02.csv", "farnsPET_r03.csv", "farnsPET_r04.csv", "farnsPET_r05.csv",
       "farnsPET_r06.csv", "farnsPET_r07.csv", "farnsPET_r08.csv", "farnsPET_r09.csv", "farnsPET_r10L.csv", 
       "farnsPET_r10U.csv", "farnsPET_r11.csv", "farnsPET_r12.csv", "farnsPET_r13.csv", "farnsPET_r14.csv", 
       "farnsPET_r15.csv", "farnsPET_r16.csv", "farnsPET_r17.csv", "farnsPET_r18.csv"]

farns = np.zeros(12 * 109951)
farns.shape = (109951, 12)

count = 0
for ii in range(len(fns)):
    farns_v10_df = pd.read_csv(filepath_or_buffer=farns_v10_path + fns[ii])
    foo = farns_v10_df.values
    nloc_hru = len(farns_v10_df.columns) - 1
    
    for jj in range(12):
        for kk in range(nloc_hru):
            ll = kk + count
            farns[ll,jj] = foo[jj,kk+1]
            
    count += nloc_hru
#    farns_v10_df['index'] = farns_v10_df['index'].apply(np.int)

print(count)

In [ ]:
# do the mapping
# there are 109951 HRUs in v1.0

nfeat = len(shape_nhm_id)

# imon goes from 0 to 11
imon = 6

pet = np.zeros(nfeat)
for ii in range(nfeat):
    ihru = shape_nhm_id[ii]-1
#    print("feature ", ii, " has hru_id ", ihru, " month ", imon, " value is ", farns[ihru,imon])
    pet[ii] = farns[ihru,imon]


In [ ]:
hrus_v10["pet"] = pet
hrus_v10.head()

In [ ]:
# plot all HRUs in white (background color)
f, ax = plt.subplots(1, figsize=(12, 12))
#hrus.plot(color="white", ax=ax)

# df.dropna(thresh=2)

# plot the HRUs with valid temperature (non-nan) with their color ramp colors
hrus_v10.plot(column="pet", cmap='coolwarm', ax=ax, legend=True)

In [ ]:
print(farns)
print(farns.shape)

In [ ]:
# Create the pandas DataFrame with the farnsworth pet values. This is for writing a csv file.
pet_df = pd.DataFrame(farns, columns = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"])
pet_df["hruid_v10"] = range(1, 109951 + 1)

# This moves the "dates" column to the first position, better for the CSV.
cols = list(pet_df.columns)
cols = [cols[-1]] + cols[:-1]
pet_df = pet_df[cols]

pet_df.head()

pet_df.to_csv(path_or_buf=farns_v10_path + "farns_gf_1_0_mm_per_month.csv")